<a href="https://colab.research.google.com/github/emmagao1/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [40.7 kB]
Get:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InR

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-12-13 02:08:19--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.99MB/s    in 0.2s    

2020-12-13 02:08:19 (4.99 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [7]:
# Create the vine_table DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3O9SGZBVQBV76|          5|            0|          0|   N|                Y|
| RKH8BNC3L5DLF|          5|            0|          0|   N|                Y|
|R2HLE8WKZSU3NL|          2|            1|          1|   N|                Y|
|R31U3UH5AZ42LL|          5|            0|          0|   N|                Y|
|R2SV659OUJ945Y|          4|            0|          0|   N|                Y|
| RA51CP8TR5A2L|          5|            0|          0|   N|                Y|
| RB2Q7DLDN6TH6|          5|            1|          1|   N|                Y|
|R2RHFJV0UYBK3Y|          1|            1|          5|   N|                N|
|R2Z6JOQ94LFHEP|          5|            1|          2|   N|                Y|
| RX27XIIWY5JPB|          4|            0|          0|   N|     

In [9]:
# Filter the data and create a new dataframe to retrieve all of the rows where the total_votes column is equal or greater than 20 
vine_df_20 = vine_df.filter(vine_df["total_votes"] >= "20")
vine_df_20.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|
| R6F9VY91ADPLA|          1|            8|         30|   N|                N|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|
|R2D8IMBVX3XCLF|          1|           14|         20|   N|                Y|
|R25UD9TA63L3Q8|          5|           25|         27|   N|     

In [10]:
# Filter the new dataframe and create new dataframe to retrieve all rows where the number of helpful_votes/total votes is equal to or greater than 50% 
new_vine_df = vine_df_20.withColumn("vote_calc", vine_df_20["helpful_votes"]/ vine_df_20["total_votes"]).filter("vote_calc>=0.5")
new_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|         vote_calc|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R14W2VCHHK5V7W|          5|           19|         20|   N|                Y|              0.95|
|R1S3T57O3OYT5S|          5|           19|         20|   N|                Y|              0.95|
|R1BTWIBLYYVOV7|          5|           30|         30|   N|                Y|               1.0|
|R3PXNV89DFIXKV|          5|           35|         37|   N|                Y|0.9459459459459459|
|R2ZF9NYVT3J7D6|          5|           19|         22|   N|                Y|0.8636363636363636|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X8FZRUOS8R8C|          4|           25|         26|   N|                Y|0.9615384615384616|
|R2D8IMBVX3XCLF|          1|  

In [11]:
# Filter the dataframe that retrieves all of the rows where a review was written as part of the vine program (paid) vine == "Y"
total_vine_reviews = new_vine_df.filter(new_vine_df["vine"] == "Y").count()
total_vine_reviews

47

In [12]:
# Filter the dataframe that retrieves all of the rows where a review was NOT written as part of the vine program (unpaid) vine == "N"
total_unpaid_reviews = new_vine_df.filter(new_vine_df["vine"] == "N").count()
total_unpaid_reviews

8362

In [13]:
# Determine the total number of reviews 
total_reviews = new_vine_df.count()
total_reviews

8409

In [14]:
# Determine the total number of five star reviews 
total_five_star_reviews = new_vine_df.filter(new_vine_df["star_rating"] == 5).count()
total_five_star_reviews

4347

In [15]:
# Determine the percentage of 5 star reviews for paid reviews (vine = Y)
total_paid_five_star = new_vine_df.filter((new_vine_df["vine"] == "Y")&(new_vine_df["star_rating"] == 5)).count()
percent_five_star_paid = total_paid_five_star/total_vine_reviews * 100
percent_five_star_paid

31.914893617021278

In [16]:
total_unpaid_five_star = new_vine_df.filter((new_vine_df["vine"] == "N")&(new_vine_df["star_rating"] == 5)).count()
percent_five_star_unpaid = total_unpaid_five_star/total_unpaid_reviews * 100
percent_five_star_unpaid

51.80578808897393

In [17]:
# Total number of unpaid (vine == N) 5 star reviews
total_unpaid_five_star

4332

In [18]:
# Total number of paid (vine == Y) 5 star reviews
total_paid_five_star

15